In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import time


In [ ]:
with open('tesco_list.txt','r') as f:
    place_list = f.readlines()

In [ ]:
for i in range(len(place_list)):
    place_list[i] = place_list[i].replace('\n','').replace(' ','-').lower()

In [ ]:
'bishops-stortford' in place_list

In [ ]:
import time
my_dict = {}
driver = webdriver.Chrome()
for place in place_list:
    driver.get(f'https://www.tesco.com/store-locator/{place}')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tag = 'address'
    attributes ={'class':'Address-content'}
    table_soup = soup.find_all(tag, attributes)
    my_list = []
    for i in range(len(table_soup)):
        address_soup = table_soup[i].find_all('span')
        my_string = ' '.join([j.text for j in address_soup])
        my_list.append(my_string)
    
    my_dict[f'{place}'] = my_list

In [ ]:
with open('tesco_dict.json','w') as f:
    json.dump(my_dict, f)

In [ ]:
for shop in ['asda','sainsbury-s','morrisons']:

    with open(f'{shop}.txt','r') as f:
        place_list = f.readlines()
    
    for i in range(len(place_list)):
        place_list[i] = place_list[i].replace('\n','').replace(' ','-').replace("'",'-').lower()

    my_dict = {}
    for place in place_list:
        driver.get(f'https://www.catalogueoffers.co.uk/stores/{shop}/locations/{place}')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tag = 'div'
        attributes ={'class':'location-name'}
        table_soup = soup.find_all(tag, attributes)

        my_list=[]
        for i in table_soup:
            address = i.text
            my_list.append(address.replace('\n','').lstrip().rstrip())
        my_dict[f'{place}'] = my_list

    with open(f'{shop}_dict.json','w') as f:
        json.dump(my_dict, f)

In [28]:
with open('tesco_dict.json','r') as f:
   data = json.load(f)

In [44]:
import pandas as pd
df = pd.DataFrame(columns = ['Location', 'Store_Count'])
df['Location'] = [location for location in data]
df['Store_Count'] = [len(data[location]) for location in data]
df

,Location,Store_Count
0,abbots-langley,1
1,aberdare,1
2,aberdeen,10
3,aberdeenshire,1
4,abergavenny,1
...,...,...
978,yateley,1
979,yeovil,4
980,york,9
981,yr-wyddgrug,1


In [42]:
df_sorter = pd.read_csv('location_region.csv')
df_sorter['Location'] = [df_sorter['Location'][i].replace(' ','-').lower() for i in range(df_sorter.shape[0])]


In [45]:
df_merge = pd.merge(df,df_sorter, on = 'Location', how = 'inner')
df_final = df_merge[['Region', 'Store_Count']].groupby('Region').sum()

,Store_Count
Region,
East,266
East Midlands,113
Isle of Man,1
London,447
North East,54
North West,220
Northern Ireland,53
Scotland,211
South East,411


In [ ]:
df_final.to_csv('tesco_locations.csv')